このスクリプトは[言語処理学会年次大会発表論文集](http://www.anlp.jp/resource/annual_meeting.html)の各年のプログラムのHTMLを受け取り、JSONファイルにタイトル (title)、著者リスト (author_list)などをキーとするJSONファイルを出力します。

* HTMLファイルはすべてUTF-8にエンコードされている（変換済み）とします。
* 元HTMLの構造が一部壊れている部分（開始タグに対応する終了タグがないなど）は、ほぼ手作業で修正します。
   * この修正結果も公開したいのですが、無許諾での著作物再配布になってしまいそうなので未公開です。
   * 最近数年の講演プログラムは修正が不要なようです。
* 講演プログラムを構成するHTMLテーブルに一定のパターンがあることを利用してアドホックに抽出しています。
* 招待講演は今のところ未対応とします。

In [1]:
import bs4
import json
import codecs

table = []
for y in xrange(1995, 2016):
    f = 'anlp%d.html' % y
    soup = bs4.BeautifulSoup(open(f), 'html.parser')
    for p in soup.find_all('td', {'class': ['paper_id', 'pid']}):
        e = p.parent
        try:
            title = e.find(['span', 'a'], class_='title').get_text().strip()
            author = e.find_next_sibling('tr').get_text().strip()
            pid = p.get_text()
            if pid and len(pid) > 0:
                table.append({
                        'year': y,
                        'pid': '%d/%s' % (y, pid),
                        'title': title,
                        'author_list': author,
                        'url': 'http://www.anlp.jp/proceedings/annual_meeting/%d/pdf_dir/%s.pdf' % (y, pid),
                        'program_url': 'http://www.anlp.jp/proceedings/annual_meeting/%d/index.html' % y})
            else:
                print('# skipped: %s: %s' % (f, p.parent))
        except AttributeError:
            print('# error: %s: %s' % (f, p.parent))
json.dump(table, codecs.open('history.json', 'w', encoding='utf-8'), indent=2, ensure_ascii=False)

# error: anlp1995.html: <tr><td class="paper_id">B1-3</td>
<td>(発表取り消し)</td></tr>
# error: anlp1995.html: <tr><td class="paper_id">B4-5</td>
<td><a href="pdf_dir/B4-5.pdf"><span class="title">カタカナ表記からの英訳推定による専門用語辞書作成</span></a><img alt="PDF" src="pdf_icon.gif"/></td></tr>
# error: anlp1999.html: <tr><td class="paper_id">C2-4</td>
<td>(発表取り消し)</td></tr>
# skipped: anlp2006.html: <tr><td class="pid"><a name="I1-1"></a></td>
<td><span class="title">人間の統語処理---内部言語と他の要因の相互作用</span> (p.1)</td></tr>
# skipped: anlp2006.html: <tr><td class="pid"><a name="I1-2"></a></td>
<td><span class="title">話し言葉と書き言葉</span> (pp.2-3)</td></tr>
# error: anlp2006.html: <tr><td class="pid"><a name="A2-4">A2-4</a></td>
<td>(発表取消)</td></tr>
# error: anlp2006.html: <tr><td class="pid"><a name="C3-4">C3-4</a></td>
<td>(発表取消)</td></tr>
# error: anlp2006.html: <tr><td class="pid"><a name="P7-6">P7-6</a></td>
<td>(発表取消)</td></tr>
# skipped: anlp2007.html: <tr><td class="pid"><a name="I1-1"></a></td>
<td><span class="t